***
![Python_logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)


   # **Cortex Game: Reto SAS**
***

## **Connect to SASPy**

In [1]:
#from platform import python_version
#print (python_version())

# pip install saspy

#import saspy, os
#print(saspy.__file__.replace('__init__.py', 'sascfg_personal.py'))

In [2]:
import saspy
sas_session = saspy.SASsession()
sas_session
;

Using SAS Config named: oda
Error trying to read authinfo file:C:\Users\hsrc1\_authinfo
[Errno 2] No such file or directory: 'C:\\Users\\hsrc1\\_authinfo'
Did not find key oda in authinfo file:C:\Users\hsrc1\_authinfo

Please enter the OMR user id: a01422876@tec.mx
Please enter the password for OMR user : ········
SAS Connection established. Subprocess id is 88956



''

***
## Connect to Cortex Data Sets

In [3]:
%%SAS sas_session
libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';

## Transform cloud SAS dataset to Python dataframe (pandas)


> **For reference**:

> 1. [Pandas library](https://pandas.pydata.org/docs/user_guide/index.html)

> 2. [sklearn.model_selection for data partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)


In [4]:
import pandas as pd
import numpy as np

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

# Merge_Data

In [5]:
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge.sample(2)

ID LastName FirstName  Woman   Age    Salary    Education  \
79302   2079303.0   MILLER     FLOYD    0.0  60.0  145700.0  High School   
524318  2524319.0   JUAREZ      MARY    1.0  58.0   20600.0  High School   

            City  SeniorList  NbActivities  ...  Frequency  Seniority  \
79302       City         9.0           0.0  ...        1.0        3.0   
524318  Suburban         1.0           0.0  ...        NaN        NaN   

        TotalGift  MinGift  MaxGift  GaveLastYear  AmtLastYear  Contact  \
79302        20.0     20.0     20.0           0.0          0.0      0.0   
524318        NaN      NaN      NaN           0.0          0.0      0.0   

        GaveThisYear  AmtThisYear  
79302            0.0          0.0  
524318           0.0          0.0  

[2 rows x 22 columns]

## Treat Missing Values

> Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.


In [6]:
data_merge.dtypes

ID              float64
LastName         object
FirstName        object
Woman           float64
Age             float64
Salary          float64
Education        object
City             object
SeniorList      float64
NbActivities    float64
Referrals       float64
Recency         float64
Frequency       float64
Seniority       float64
TotalGift       float64
MinGift         float64
MaxGift         float64
GaveLastYear    float64
AmtLastYear     float64
Contact         float64
GaveThisYear    float64
AmtThisYear     float64
dtype: object

In [7]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)

data_merge['Age2'] = np.where(data_merge['Age']<= 46, 1, 0)
data_merge['Pct'] = round(((data_merge['TotalGift']/data_merge['Salary'])+1),10)
data_merge.replace([np.inf,-np.inf],np.nan, inplace = True)
data_merge[['Pct']] = data_merge[['Pct']].fillna(value=0)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)
data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)

data_merge = data_merge.loc[data_merge['TotalGift'] < 8000]
print(data_merge.shape)
#data_merge['AmtThisYear'].describe()


print(data_merge.isnull().sum())

data_merge.sample(3)

(999914, 24)
ID               0
LastName        23
FirstName        4
Woman            0
Age              0
Salary           0
Education        0
City             0
SeniorList       0
NbActivities     0
Referrals        0
Recency          0
Frequency        0
Seniority        0
TotalGift        0
MinGift          0
MaxGift          0
GaveLastYear     0
AmtLastYear      0
Contact          0
GaveThisYear     0
AmtThisYear      0
Age2             0
Pct              0
dtype: int64


ID LastName FirstName  Woman   Age   Salary  Education  City  \
993477  2993478.0   MANUEL      MARY    1.0  71.0  13900.0          2     3   
364148  2364149.0  RADFORD  MARGARET    1.0  16.0   2700.0          2     2   
420790  2420791.0   MEJIAS      JEFF    0.0  27.0  53000.0          1     3   

        SeniorList  NbActivities  ...  TotalGift  MinGift  MaxGift  \
993477         0.0           0.0  ...        0.0      0.0      0.0   
364148         5.0           1.0  ...      195.0     20.0    175.0   
420790         0.0           0.0  ...        0.0      0.0      0.0   

        GaveLastYear  AmtLastYear  Contact  GaveThisYear  AmtThisYear  Age2  \
993477           0.0          0.0      0.0           0.0          0.0     0   
364148           0.0          0.0      0.0           0.0          0.0     1   
420790           1.0         10.0      0.0           0.0          0.0     1   

             Pct  
993477  1.000000  
364148  1.072222  
420790  1.000000  

[3 rows x 24 columns]

In [8]:
#data_merge['TotalGift'].describe()

In [9]:
#pd.options.display.max_rows = 4000

print(data_merge['Pct'].unique())
data_merge['Pct'].describe()

[1.00942164 1.         1.00076253 ... 1.00159262 1.02593857 1.00039646]


count    999914.000000
mean          1.001140
std           0.035903
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000429
max          13.000000
Name: Pct, dtype: float64

In [10]:
pd.options.display.max_rows = 4000

data_merge.groupby(['Age','Education']).TotalGift.mean()

Age   Education
16.0  1            31.103018
      2            35.299226
      3            21.984925
17.0  1            34.804691
      2            32.671865
      3            39.554974
18.0  1            31.209710
      2            35.731465
      3            65.230769
19.0  1            33.260202
      2            33.030424
      3            36.213873
20.0  1            33.543814
      2            33.619243
      3            30.089286
21.0  1            33.078663
      2            39.165038
      3            61.607143
22.0  1            31.400340
      2            32.667432
      3            41.237113
23.0  1            32.827715
      2            33.170881
      3            26.994819
24.0  1            30.673506
      2            37.080028
      3            28.061224
25.0  1            45.169368
      2            41.974857
      3            25.072727
26.0  1            47.326998
      2            36.884907
      3            27.670455
27.0  1            46.75839

## Data Partition

In [11]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split

#data_merge = data_merge.sample(120000)
print(data_merge.shape)
train, validation = train_test_split(data_merge, test_size = 0.4, random_state=12345) 

#train.head()
train.sample(2)

(999914, 24)


ID LastName FirstName  Woman   Age   Salary  Education  City  \
915523  2915524.0  VASQUEZ      JOSE    0.0  34.0  16300.0          1     4   
255189  2255190.0     KEHL   WILLARD    0.0  51.0  66200.0          1     1   

        SeniorList  NbActivities  ...  TotalGift  MinGift  MaxGift  \
915523        10.0           0.0  ...        0.0      0.0      0.0   
255189         7.0           0.0  ...       55.0     10.0     30.0   

        GaveLastYear  AmtLastYear  Contact  GaveThisYear  AmtThisYear  Age2  \
915523           0.0          0.0      0.0           0.0          0.0     1   
255189           0.0          0.0      0.0           1.0         20.0     0   

             Pct  
915523  1.000000  
255189  1.000831  

[2 rows x 24 columns]

# Model: The Champion

### **XG BOOST**

In [12]:
import xgboost as xgb

In [13]:
X_train = train[['NbActivities','Contact','Woman','Education','Referrals','Frequency', 'Seniority','SeniorList','Age','Salary','TotalGift']]
Y_train = train['AmtThisYear']
X_valid = validation[['NbActivities','Contact','Woman','Education','Referrals','Frequency', 'Seniority','SeniorList','Age','Salary','TotalGift']]
Y_valid = validation['AmtThisYear']

regr = xgb.XGBRegressor(booster = 'dart',tree_method = 'approx',n_estimators=200, max_depth=3, eta=0.1,subsample=0.8,colsample_bytree=0.5)
regr.fit(X_train,Y_train)

regr_predict=regr.predict(X_valid)

#you can change the criteria

from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

15.037130165761795
8792.64680948917
93.76911436869375


***
# ROUND 2: FIGHT

In [14]:
import pandas as pd

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

## Merge the Data

In [15]:
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
#data_merge.head()
data_merge.sample(2)

ID LastName FirstName  Woman   Age   Salary  \
109866  2109867.0  MORILLO     KERRI    1.0  75.0  17500.0   
403609  2403610.0   BARNES     APRIL    1.0  20.0  11000.0   

                   Education      City  SeniorList  NbActivities  ...  \
109866           High School  Downtown         5.0           1.0  ...   
403609  University / College     Rural         6.0           0.0  ...   

        Frequency  Seniority  TotalGift  MinGift  MaxGift  GaveLastYear  \
109866        1.0        3.0       30.0     30.0     30.0           0.0   
403609        1.0        4.0       75.0     75.0     75.0           0.0   

        AmtLastYear  Contact  GaveThisYear  AmtThisYear  
109866          0.0      0.0           0.0          0.0  
403609          0.0      0.0           0.0          0.0  

[2 rows x 22 columns]

## Treating Missing Values

Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.

In [16]:
data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)

data_merge['Age2'] = np.where(data_merge['Age']<= 46, 1, 0)
data_merge['Pct'] = round(((data_merge['TotalGift']/data_merge['Salary'])+1),10)
data_merge.replace([np.inf,-np.inf],np.nan, inplace = True)
data_merge[['Pct']] = data_merge[['Pct']].fillna(value=0)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)
data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)


print(data_merge.isnull().sum())

data_merge.sample(3)

ID               0
LastName        23
FirstName        4
Woman            0
Age              0
Salary           0
Education        0
City             0
SeniorList       0
NbActivities     0
Referrals        0
Recency          0
Frequency        0
Seniority        0
TotalGift        0
MinGift          0
MaxGift          0
GaveLastYear     0
AmtLastYear      0
Contact          0
GaveThisYear     0
AmtThisYear      0
Age2             0
Pct              0
dtype: int64


ID  LastName FirstName  Woman   Age   Salary  Education  City  \
690793  2690794.0      BANH   WILLIAM    0.0  25.0  20800.0          2     4   
518994  2518995.0  WHELCHEL   TIMOTHY    0.0  62.0  26000.0          1     3   
956976  2956977.0    MCAVOY      PAUL    0.0  52.0  13100.0          2     3   

        SeniorList  NbActivities  ...  TotalGift  MinGift  MaxGift  \
690793         1.0           0.0  ...        0.0      0.0      0.0   
518994         2.0           1.0  ...      100.0    100.0    100.0   
956976         8.0           1.0  ...       25.0     25.0     25.0   

        GaveLastYear  AmtLastYear  Contact  GaveThisYear  AmtThisYear  Age2  \
690793           0.0          0.0      0.0           0.0          0.0     1   
518994           0.0          0.0      0.0           0.0          0.0     0   
956976           0.0          0.0      0.0           0.0          0.0     0   

             Pct  
690793  1.000000  
518994  1.003846  
956976  1.001908  

[3 rows x 24 columns]

## Decision Tree Model

In [18]:

from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.4, random_state=12345) 

#train.head()
train.sample(2)

ID    LastName  FirstName  Woman   Age   Salary  Education  \
321248  2321249.0  HUNSBERGER        ROY    0.0  47.0  33300.0          2   
155735  2155736.0       BALAN  CATHERINE    1.0  34.0  23000.0          2   

        City  SeniorList  NbActivities  ...  TotalGift  MinGift  MaxGift  \
321248     2         2.0           0.0  ...       75.0     75.0     75.0   
155735     4        10.0           0.0  ...        0.0      0.0      0.0   

        GaveLastYear  AmtLastYear  Contact  GaveThisYear  AmtThisYear  Age2  \
321248           0.0          0.0      1.0           0.0          0.0     0   
155735           0.0          0.0      0.0           0.0          0.0     1   

             Pct  
321248  1.002252  
155735  1.000000  

[2 rows x 24 columns]

In [19]:
from sklearn.tree import DecisionTreeClassifier

X_train = train[['NbActivities','Contact','Woman','Education','Referrals','Frequency', 'Seniority','SeniorList','Age','Salary','TotalGift']]
Y_train = train['AmtThisYear']
X_valid = validation[['NbActivities','Contact','Woman','Education', 'Referrals','Frequency', 'Seniority','SeniorList','Age','Salary','TotalGift']] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeClassifier(max_depth=3,criterion="entropy").fit(X_train,Y_train)
DT_predict_proba = DT_model.predict_proba(X_valid) #Predictions on Testing data
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
# Probabilities for each class
DT_probs = DT_model.predict_proba(X_valid)[:, 1]
print(DT_probs)

[0.02189186 0.02189186 0.04894784 ... 0.01427146 0.09640847 0.01427146]


# XGBoost: Classify this...

In [38]:
from xgboost import XGBClassifier

X_train = train[['NbActivities','Contact','Woman','Education','Referrals','Frequency', 'Seniority','SeniorList','Age','Salary','TotalGift']]
Y_train = train['AmtThisYear']
X_valid = validation[['NbActivities','Contact','Woman','Education', 'Referrals','Frequency', 'Seniority','SeniorList','Age','Salary','TotalGift']] 
Y_valid = validation['AmtThisYear']

DT_model = XGBClassifier(n_estimators=200, eta = 0.1, max_depth = 3, subsample = 0.7,colsample_bytree = 0.6, verbosity = 0).fit(X_train,Y_train)
DT_predict_proba = DT_model.predict_proba(X_valid) #Predictions on Testing data
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
# Probabilities for each class
DT_probs = DT_model.predict_proba(X_valid)[:, 1]
print(DT_probs)

C:\Users\hsrc1\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0.00932322 0.04518163 0.05031437 ... 0.02194701 0.0809491  0.01245307]


In [39]:
#you can change the criteria
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix


confusion_matrix = confusion_matrix(Y_valid, DT_predict)
print(confusion_matrix)


[[340153      6      0 ...      0      0      0]
 [  9250      8      0 ...      0      0      0]
 [  1018      3      0 ...      0      0      0]
 ...
 [     7      0      0 ...      0      0      0]
 [     3      0      0 ...      0      0      0]
 [     9      0      0 ...      0      0      0]]


In [40]:
from sklearn.metrics import classification_report
print(classification_report(Y_valid, DT_predict))

C:\Users\hsrc1\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.85      1.00      0.92    340213
        10.0       0.19      0.00      0.00      9310
        15.0       0.00      0.00      0.00      1023
        20.0       0.28      0.01      0.01     17143
        25.0       0.00      0.00      0.00      6511
        30.0       0.00      0.00      0.00      5730
        40.0       0.00      0.00      0.00      4673
        50.0       0.00      0.00      0.00      4675
        60.0       0.00      0.00      0.00       315
        70.0       0.00      0.00      0.00       275
        75.0       0.00      0.00      0.00      2521
        80.0       0.00      0.00      0.00       272
        90.0       0.00      0.00      0.00       304
       100.0       0.00      0.00      0.00      2412
       120.0       0.00      0.00      0.00       236
       125.0       0.00      0.00      0.00       212
       150.0       0.00      0.00      0.00       983
       175.0       0.00    

C:\Users\hsrc1\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hsrc1\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Scoring New Data

### Prepare data for scoring

In [22]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='SCORE_RD2_NOCONTACT',
libref='cortex'
)

 ### Score new data based on your champion model
 
 Pick your champion model from previous steps and use it to predict next year donations. 
 
 In this case, the Decision Tree model performed better than the Logistic Regression model based on the AUC criterion.

### Predict 'probability of giving' for members who were contacted

In [23]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

data_merge = scoring_data_contact

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)

data_merge['Age2'] = np.where(data_merge['Age']<= 46, 1, 0)
data_merge['Pct'] = round(((data_merge['TotalGift']/data_merge['Salary'])+1),10)
data_merge.replace([np.inf,-np.inf],np.nan, inplace = True)
data_merge[['Pct']] = data_merge[['Pct']].fillna(value=0)

data_merge = data_merge.loc[data_merge['TotalGift'] < 9000]
print(data_merge.shape)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)
data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)


scoring_data_contact = data_merge

#scoring_data_contact.head()

X = scoring_data_contact[['NbActivities','Contact','Woman','Education','Referrals','Frequency', 'Seniority','SeniorList','Age','Salary','TotalGift']]
DT_predict_contact=DT_model.predict_proba(X)[:,1]
scoring_data_contact['Prediction_prob'] = DT_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction_prob']]
scoring_data_contact = scoring_data_contact.rename({'Prediction_prob': 'ProbContact'}, axis=1) 
scoring_data_contact.head()

(999904, 22)


C:\Users\hsrc1\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


ID  ProbContact
0  2000001.0     0.061077
1  2000002.0     0.096408
2  2000003.0     0.096408
3  2000004.0     0.044704
4  2000005.0     0.096408

### Predict 'probability of giving' for members who were not contacted

In [24]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

data_merge = scoring_data_nocontact

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)

data_merge['Age2'] = np.where(data_merge['Age']<= 46, 1, 0)
data_merge['Pct'] = round(((data_merge['TotalGift']/data_merge['Salary'])+1),10)
data_merge.replace([np.inf,-np.inf],np.nan, inplace = True)
data_merge[['Pct']] = data_merge[['Pct']].fillna(value=0)

data_merge = data_merge.loc[data_merge['TotalGift'] < 9000]
print(data_merge.shape)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)
data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)


scoring_data_nocontact = data_merge
#scoring_data_contact.head()

X = scoring_data_nocontact[['NbActivities','Contact','Woman','Education','Referrals','Frequency', 'Seniority','SeniorList','Age','Salary','TotalGift']]
DT_predict_nocontact=DT_model.predict_proba(X)[:,1]
scoring_data_nocontact['Prediction_prob'] = DT_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction_prob']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction_prob': 'ProbNoContact'}, axis=1) 
scoring_data_nocontact.head()

(999904, 22)


C:\Users\hsrc1\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


ID  ProbNoContact
0  2000001.0       0.021892
1  2000002.0       0.014271
2  2000003.0       0.014271
3  2000004.0       0.014271
4  2000005.0       0.014271

In [25]:
result_Prob = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Prob.sort_values(by=['ID'], inplace=True)
result_Prob.sample(10)

ID  ProbContact  ProbNoContact
620734  2620793.0     0.044704       0.048948
324122  2324153.0     0.096408       0.014271
156378  2156391.0     0.083744       0.014271
773789  2773866.0     0.083744       0.014271
181548  2181564.0     0.096408       0.027818
901570  2901659.0     0.061077       0.014271
96594   2096600.0     0.096408       0.014271
846656  2846739.0     0.044704       0.014271
398635  2398672.0     0.096408       0.014271
316315  2316346.0     0.061077       0.021892

## Exporting Results to a CSV File

In [34]:
result_Prob.to_csv('Round2_Output_prob.csv', index=False)

***
# ROUND 2: FATALITY

## Scoring New Data

### Prepare data for scoring

In [27]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

### Predict 'amount given' for members who were contacted

In [28]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

data_merge = scoring_data_contact

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)

data_merge['Age2'] = np.where(data_merge['Age']<= 46, 1, 0)
data_merge['Pct'] = round(((data_merge['TotalGift']/data_merge['Salary'])+1),10)
data_merge.replace([np.inf,-np.inf],np.nan, inplace = True)
data_merge[['Pct']] = data_merge[['Pct']].fillna(value=0)

data_merge = data_merge.loc[data_merge['TotalGift'] < 9000]
print(data_merge.shape)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)
data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)


scoring_data_contact = data_merge
#scoring_data_contact.head()

X = scoring_data_contact[['NbActivities','Contact','Woman','Education','Referrals','Frequency', 'Seniority','SeniorList','Age','Salary','TotalGift']]

regr_predict_contact=regr.predict(X)

scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

(999904, 22)


C:\Users\hsrc1\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


ID  AmtContact
0  2000001.0   77.213615
1  2000002.0   38.119888
2  2000003.0   42.919670
3  2000004.0   12.345148
4  2000005.0   41.324638

### Predict 'amount given' for members who were not contacted

In [29]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

data_merge = scoring_data_nocontact

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)

data_merge['Age2'] = np.where(data_merge['Age']<= 46, 1, 0)
data_merge['Pct'] = round(((data_merge['TotalGift']/data_merge['Salary'])+1),10)
data_merge.replace([np.inf,-np.inf],np.nan, inplace = True)
data_merge[['Pct']] = data_merge[['Pct']].fillna(value=0)

data_merge = data_merge.loc[data_merge['TotalGift'] < 9000]
print(data_merge.shape)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)
data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)


scoring_data_nocontact = data_merge

#scoring_data_nocontact.head()

X = scoring_data_nocontact[['NbActivities','Contact','Woman','Education','Referrals','Frequency', 'Seniority','SeniorList','Age','Salary','TotalGift']]

regr_predict_nocontact=regr.predict(X)

scoring_data_nocontact['Prediction'] = regr_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

(999904, 22)


C:\Users\hsrc1\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


ID  AmtNoContact
0  2000001.0     17.670300
1  2000002.0     12.334988
2  2000003.0      6.697234
3  2000004.0      2.234595
4  2000005.0      5.839808

In [30]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

ID  AmtContact  AmtNoContact
0  2000001.0   77.213615     17.670300
1  2000002.0   38.119888     12.334988
2  2000003.0   42.919670      6.697234

In [35]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

## Exporting Results to a CSV File

In [36]:
import pandas as pd

NB = 160000

probabilities = pd.read_csv("Round2_Output_prob.csv")
amounts = pd.read_csv("Round2_Output_amt.csv")

def Calc_Uplift(raw_data):
    return ((raw_data['AmtContact']*raw_data['ProbContact']) - (raw_data['AmtNoContact']*raw_data['ProbNoContact']))

raw_submission = pd.merge(probabilities, amounts, on=["ID"], how="right")
raw_submission["Uplift"] = raw_submission.apply(lambda row: Calc_Uplift(row), axis=1)

# Sorting data by descending Uplift value
raw_submission.sort_values(by=['Uplift'], ascending=False, inplace=True)


In [37]:
# Export the final csv file

submission = raw_submission.head(NB)

submission.to_csv('Round2 Output final.csv', index=False)
